# Import

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown

sns.set_theme(style="darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "Apple-Quality"
COLAB = 'google.colab' in sys.modules

DEBUG = False
SEED = 666

In [2]:
COLAB = 'google.colab' in sys.modules

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)
else:
  ROOT = "./"

def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Load Dataset

In [3]:
df = pd.read_pickle(f"{ROOT}/data/apple.pickle")

In [4]:
print(df.shape)
df.head()

(4000, 8)


,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Target
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,0
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,0
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,1
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,0
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,0


# Baseline Model

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

## Split

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('Target', axis=True)

y = df.Target


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED)
y_train.value_counts()


Target
0    1614
1    1586
Name: count, dtype: int64

## Train

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

classifiers = {
    "KNN": KNeighborsClassifier(),
    "KNN(3)": KNeighborsClassifier(3),
    "DT": DecisionTreeClassifier(),
    "DT(max_depth=5)": DecisionTreeClassifier(max_depth=5),
    "RF": RandomForestClassifier()
}


for name, model in classifiers.items():

    model.fit(X_train, y_train)

    # Scoring on SEEN data - effectively "useless"
    y_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_pred)

    # Scoring on UNSEEN data - important
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    print(f"{name:20s} accuracy\ttrain = {train_accuracy:.2%} \ttest = {test_accuracy:.2%}")
    
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("\nAfter scaling\n")

for name, model in classifiers.items():

    model.fit(X_train, y_train)

    # Scoring on SEEN data - effectively "useless"
    y_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_pred)

    # Scoring on UNSEEN data - important
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    print(f"{name:20s} accuracy\ttrain = {train_accuracy:.2%} \ttest = {test_accuracy:.2%}")

KNN                  accuracy	train = 92.69% 	test = 91.12%
KNN(3)               accuracy	train = 94.38% 	test = 90.25%
DT                   accuracy	train = 100.00% 	test = 79.00%
DT(max_depth=5)      accuracy	train = 76.84% 	test = 74.12%
RF                   accuracy	train = 100.00% 	test = 90.50%

After scaling

KNN                  accuracy	train = 92.81% 	test = 91.25%
KNN(3)               accuracy	train = 94.06% 	test = 90.00%
DT                   accuracy	train = 100.00% 	test = 77.88%
DT(max_depth=5)      accuracy	train = 76.84% 	test = 74.00%
RF                   accuracy	train = 100.00% 	test = 90.50%


## Test

## Resultados

KNN                  accuracy	train = 92.69% 	test = 91.12%
KNN(3)               accuracy	train = 94.38% 	test = 90.25%
DT                   accuracy	train = 100.00% 	test = 79.00%
DT(max_depth=5)      accuracy	train = 76.84% 	test = 74.12%
RF                   accuracy	train = 100.00% 	test = 90.50%

After scaling

KNN                  accuracy	train = 92.81% 	test = 91.25%
KNN(3)               accuracy	train = 94.06% 	test = 90.00%
DT                   accuracy	train = 100.00% 	test = 77.88%
DT(max_depth=5)      accuracy	train = 76.84% 	test = 74.00%
RF                   accuracy	train = 100.00% 	test = 90.50%


# Questões

**Pergunta:** Qual é a diferença entre dados de treinamento e dados de teste ao treinar um modelo de classificação?


Os dados de treinamento e os dados de teste são usados ao treinar um modelo de classificação de forma a avaliar o desempenho do modelo em diferentes conjuntos de dados.

Os dados de treinamento são usados para treinar o modelo, ou seja, para ajustar os parâmetros do modelo com base nos exemplos fornecidos. O modelo aprende com esses dados e tenta capturar os padrões e relações entre as variáveis de entrada e a variável de saída.

Os dados de teste são usados para avaliar o desempenho do modelo em dados não vistos durante o treinamento. Esses dados são usados para medir a capacidade do modelo de generalizar e fazer previsões precisas em novos exemplos. Ao avaliar o modelo com os dados de teste, podemos ter uma ideia de como o modelo se comportará em situações reais.

A diferença fundamental entre os dados de treinamento e os dados de teste é que os dados de treinamento são usados para ajustar o modelo, enquanto os dados de teste são usados para avaliar o desempenho do modelo. É importante que os dados de teste sejam independentes e representativos dos dados reais que o modelo encontrará após o treinamento.

**Pergunta:** Com base no dataset fornecido, a abordagem do treinamento do modelo deve ser `supervisionada` ou `não-supervisionada`? Por quê?


Com base no dataset fornecido, a abordagem do treinamento do modelo deve ser `supervisionada`. Isso ocorre porque o dataset possui uma variável alvo (Target) que indica a classe ou categoria a ser prevista. Nesse tipo de abordagem, o modelo é treinado usando exemplos rotulados, ou seja, cada exemplo no dataset possui uma classe ou categoria associada. O objetivo é aprender um mapeamento entre as variáveis de entrada (Size, Weight, Sweetness, Crunchiness, Juiciness, Ripeness, Acidity) e a variável alvo (Target) para fazer previsões precisas em novos exemplos não vistos durante o treinamento.